<summary><strong>Install Dependencies</strong></summary>

In [1]:
# Make sure to Run this once
!pip install pandas python-dotenv requests sentence-transformers faiss-cpu scikit-learn

<strong>Environment Setup & Configuration</strong>

In [2]:
import os
import re
import pickle
from pathlib import Path
from typing import List, Dict, Any, Tuple

import pandas as pd
import requests

from dotenv import load_dotenv
from difflib import get_close_matches

from sentence_transformers import SentenceTransformer
import faiss
from sklearn.preprocessing import normalize

In [3]:
load_dotenv()
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")

if not OPENROUTER_API_KEY:
    print("WARNING: OPENROUTER_API_KEY not found in .env file")
else:
    print("API key loaded successfully")

API key loaded successfully


In [4]:
DATA_DIR = Path("data")
CACHE_DIR = DATA_DIR / "cache"
CACHE_DIR.mkdir(parents=True, exist_ok=True)

TRADES_CSV = DATA_DIR / "trades.csv"
HOLDINGS_CSV = DATA_DIR / "holdings.csv"

EMBED_MODEL_NAME = "all-MiniLM-L6-v2"
CHUNK_EMB_FILE = CACHE_DIR / "chunk_embeddings.pkl"
CHUNKS_FILE = CACHE_DIR / "chunks.pkl"

# OpenRouter
load_dotenv()
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY", "").strip()

# Retrieval params
CHUNK_TOP_K = 5
CHUNK_MIN_SCORE = 0.35

<strong>Exploratory Data Analysis</strong>

In [5]:
# Load data
print("Loading data...")
trades = pd.read_csv(TRADES_CSV, dtype=str)
holdings = pd.read_csv(HOLDINGS_CSV, dtype=str)

print(f"Loaded {len(trades)} trades")
print(f"Loaded {len(holdings)} holdings")

Loading data...
Loaded 649 trades
Loaded 1022 holdings


In [6]:
trades = pd.read_csv(TRADES_CSV, dtype=str)
holdings = pd.read_csv(HOLDINGS_CSV, dtype=str)

print("Trades shape:", trades.shape)
print("Holdings shape:", holdings.shape)

Trades shape: (649, 31)
Holdings shape: (1022, 25)


In [7]:
display(trades.head(3))

,id,RevisionId,AllocationId,TradeTypeName,SecurityId,SecurityType,Name,Ticker,CUSIP,ISIN,...,AllocationFees,AllocationCash,PortfolioName,CustodianName,StrategyName,Strategy1Name,Strategy2Name,Counterparty,AllocationRule,IsCustomAllocation
0,3489863,2,3460886,Buy,270471,Equity,Berry Brand 4/11 Equity,NaN,NaN,NaN,...,2800,7002800,HoldCo 1,JP MORGAN SECURITIES LLC,Default,DefaultS1,DefaultS2,ABGS,Single Fund Rule - HoldCo 1,1
1,3489864,1,3460887,Sell,270471,Equity,Berry Brand 4/11 Equity,NaN,NaN,NaN,...,128.8,6999871.2,HoldCo 1,JP MORGAN SECURITIES LLC,Default,DefaultS1,DefaultS2,ABGS,Single Fund Rule - HoldCo 1,0
2,3496826,1,3462756,Sell,290063,Equity,META-US,META,30303M102,US30303M1027,...,46985.99,2553539898,HoldCo 3,CITIGROUP GLOBAL MARKETS INC.,Default,DefaultS1,DefaultS2,ABGS,Single Fund Rule - HoldCo 3,0


In [8]:
display(holdings.head(3))

,AsOfDate,OpenDate,CloseDate,ShortName,PortfolioName,StrategyRefShortName,Strategy1RefShortName,Strategy2RefShortName,CustodianName,DirectionName,...,StartPrice,Price,StartFXRate,FXRate,MV_Local,MV_Base,PL_DTD,PL_QTD,PL_MTD,PL_YTD
0,01/08/23,04/03/20,NaN,Garfield,Garfield,Default,Asset,DefaultS2,Well Prime,Long,...,96,96,1.33,1.33,568320,755865.6,92.504,10833.7294,92.504,41054.5854
1,01/08/23,04/03/20,NaN,Garfield,Garfield,Default,Asset,DefaultS2,Well Prime,Long,...,96,96,1.33,1.33,84.48,112.3584,0.0138,1.6104,0.0138,6.1027
2,01/08/23,04/03/20,NaN,Garfield,Garfield,Default,Asset,DefaultS2,Well Prime,Long,...,96,96,1.33,1.33,756000,1005480,123.0523,14411.4221,123.0523,54612.3074


In [9]:
print("Trades columns:", list(trades.columns))
print("Holdings columns:", list(holdings.columns))

Trades columns: ['id', 'RevisionId', 'AllocationId', 'TradeTypeName', 'SecurityId', 'SecurityType', 'Name', 'Ticker', 'CUSIP', 'ISIN', 'TradeDate', 'SettleDate', 'Quantity', 'Price', 'TradeFXRate', 'Principal', 'Interest', 'TotalCash', 'AllocationQTY', 'AllocationPrincipal', 'AllocationInterest', 'AllocationFees', 'AllocationCash', 'PortfolioName', 'CustodianName', 'StrategyName', 'Strategy1Name', 'Strategy2Name', 'Counterparty', 'AllocationRule', 'IsCustomAllocation']
Holdings columns: ['AsOfDate', 'OpenDate', 'CloseDate', 'ShortName', 'PortfolioName', 'StrategyRefShortName', 'Strategy1RefShortName', 'Strategy2RefShortName', 'CustodianName', 'DirectionName', 'SecurityId', 'SecurityTypeName', 'SecName', 'StartQty', 'Qty', 'StartPrice', 'Price', 'StartFXRate', 'FXRate', 'MV_Local', 'MV_Base', 'PL_DTD', 'PL_QTD', 'PL_MTD', 'PL_YTD']


In [10]:
holdings['SecurityId'].value_counts().head(20)

SecurityId
288790    115
288800     40
289892     21
277144     19
288857     16
280313     15
290030     14
272584     14
279455     14
273098     13
288798     12
279728     11
276270     11
289540     10
278822     10
288805      9
288799      9
289856      8
288808      8
288791      8
Name: count, dtype: int64

In [11]:
trades['SecurityId'].value_counts().head(20)

SecurityId
288790    33
288800    30
290081    20
288798    16
290024    12
279524    10
290087     9
279099     9
290027     8
288793     8
288792     8
271935     8
288822     8
270992     8
279728     8
288805     8
290025     8
290026     8
290067     6
288799     6
Name: count, dtype: int64

In [12]:
trades['PortfolioName'].value_counts().head(20)

PortfolioName
Redfield Accu-Fund           143
UNC Investment Fund          142
Leatherwood Trust MA         132
Optimum Holdings Partners    101
HoldCo 1                      43
CampNou Holdings              30
ClientA                       24
Platpot Fund                   9
HoldCo 11                      6
Northpoint 401K                6
Account A                      5
Account B                      3
Account D                      2
HoldCo 3                       1
HoldCo 7                       1
Account C                      1
Name: count, dtype: int64

In [13]:
holdings['PortfolioName'].value_counts().head(20)

PortfolioName
MNC Investment Fund        243
Garfield                   221
Heather                    195
Opium Holdings Partners    131
Platpot                     61
Ytum                        51
Hi Yield                    19
NPSMF1                      17
NPSMF3                      17
NPSMF2                      17
Warren Lee IG               15
Northpoint 401K             14
CoYold 1                     7
SMA-L1                       3
SMA-L2                       3
SMA-L4                       3
CoYold 11                    3
IG Corp                      1
CoYold 7                     1
Name: count, dtype: int64

In [14]:
holdings.groupby("PortfolioName")[["PL_YTD", "PL_MTD", "PL_QTD"]].count()

,PL_YTD,PL_MTD,PL_QTD
PortfolioName,,,
CoYold 1,7,7,7
CoYold 11,3,3,3
CoYold 7,1,1,1
Garfield,221,221,221
Heather,195,195,195
Hi Yield,19,19,19
IG Corp,1,1,1
MNC Investment Fund,243,243,243
NPSMF1,17,17,17


In [15]:
holdings['PortfolioName'] = holdings['PortfolioName'].str.strip().str.lower()
trades['PortfolioName'] = trades['PortfolioName'].str.strip().str.lower()

len(holdings['PortfolioName'].unique())

19

In [16]:
len(trades['PortfolioName'].unique())

16

<strong>Data Loading, Cleaning & Normalization</strong>

In [17]:
def safe_read_csv(path: Path) -> pd.DataFrame:
    if not path.exists():
        raise FileNotFoundError(f"{path} not found. Place the CSV in {path}")
    return pd.read_csv(path, dtype=str)

In [18]:
def preprocess_df(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = [c.strip() for c in df.columns]
    for c in df.columns:
        if df[c].dtype == object:
            df[c] = df[c].fillna('').astype(str).str.strip()
    return df

In [19]:
def safe_to_numeric(series: pd.Series) -> pd.Series:
    if series is None:
        return pd.Series(dtype=float)
    s = series.astype(str).fillna('').str.replace(',', '', regex=False)
    s = s.str.replace('(', '-', regex=False).str.replace(')', '', regex=False)
    return pd.to_numeric(s, errors='coerce')

print("Data preprocessed and numeric columns converted")

Data preprocessed and numeric columns converted


In [20]:
def normalize_text(text: str) -> str:
    if text is None:
        return ""
    s = str(text).lower().strip()
    s = re.sub(r'[^a-z0-9]', '', s)
    return s

In [21]:
print("Loading datafor Preprocessing...")
trades = preprocess_df(safe_read_csv(TRADES_CSV))
holdings = preprocess_df(safe_read_csv(HOLDINGS_CSV))
print(f"Loaded trades: {len(trades)} rows, holdings: {len(holdings)} rows")

for col in ['Quantity', 'Price', 'Principal', 'TotalCash', 'AllocationQTY']:
    if col in trades.columns:
        trades[col + '_num'] = safe_to_numeric(trades[col])

for col in ['Qty', 'StartQty', 'Price', 'MV_Base', 'MV_Local', 'PL_YTD', 'PL_MTD', 'PL_QTD', 'PL_DTD']:
    if col in holdings.columns:
        holdings[col + '_num'] = safe_to_numeric(holdings[col])

Loading datafor Preprocessing...
Loaded trades: 649 rows, holdings: 1022 rows


In [22]:
def get_portfolio_set(trades_df: pd.DataFrame, holdings_df: pd.DataFrame) -> List[str]:
    p1 = [p for p in trades_df['PortfolioName'].astype(str).unique() if p and p.upper() != 'NULL']
    p2 = [p for p in holdings_df['PortfolioName'].astype(str).unique() if p and p.upper() != 'NULL']
    p3 = [s for s in holdings_df.get('ShortName', pd.Series([])).astype(str).unique() if s and s.upper() != 'NULL']
    allp = sorted(list(set([x.strip() for x in (p1 + p2 + p3) if x and x.strip()])))
    return allp

ALL_PORTFOLIOS = get_portfolio_set(trades, holdings)
TICKERS = sorted([t for t in trades.get('Ticker', pd.Series([])).astype(str).unique() if t and t.upper() != 'NULL'])

print(f"Found {len(ALL_PORTFOLIOS)} unique portfolio identifiers.")
print(f"Found {len(TICKERS)} unique tickers.")

Found 41 unique portfolio identifiers.
Found 44 unique tickers.


In [23]:
def extract_entities_fuzzy(question: str, debug: bool=False) -> Dict[str, Any]:
    q = question or ""
    q_lower = q.lower()
    q_norm = normalize_text(q)
    entities: Dict[str, Any] = {}

    norms_to_portfolio = {normalize_text(p): p for p in ALL_PORTFOLIOS}
    if q_norm in norms_to_portfolio:
        entities['portfolio'] = norms_to_portfolio[q_norm]
        if debug: print("[debug] direct portfolio normalized match:", entities['portfolio'])
        return entities

    tokens = re.findall(r'[A-Za-z0-9]+', q)
    for t in tokens[::-1]:
        tn = normalize_text(t)
        if tn in norms_to_portfolio:
            entities['portfolio'] = norms_to_portfolio[tn]
            if debug: print("[debug] token portfolio match:", entities['portfolio'])
            break

    if 'portfolio' not in entities:
        token_candidates = [normalize_text(t) for t in tokens]
        portfolio_keys = list(norms_to_portfolio.keys())
        for cand in token_candidates:
            matches = get_close_matches(cand, portfolio_keys, n=1, cutoff=0.72)
            if matches:
                entities['portfolio_suggestion'] = norms_to_portfolio[matches[0]]
                if debug: print("[debug] fuzzy portfolio suggestion:", entities['portfolio_suggestion'])
                break

    ticker_match = re.findall(r'\b[A-Z]{1,5}\b', question or "")
    if ticker_match:
        for tk in ticker_match:
            if tk in TICKERS:
                entities['ticker'] = tk
                break
        if 'ticker' not in entities:
            entities['ticker'] = ticker_match[0]  

    if 'ticker' not in entities:
        for tk in TICKERS:
            if normalize_text(tk) in q_norm:
                entities['ticker'] = tk
                break

    if any(w in q_lower for w in ['buy', 'bought', 'purchase', 'purchased']):
        entities['trade_type'] = 'Buy'
    elif any(w in q_lower for w in ['sell', 'sold', 'sale']):
        entities['trade_type'] = 'Sell'

    metric_aliases = {
        'pl_ytd': ['pl ytd', 'pl_ytd', 'pl-ytd', 'ytd profit', 'yearly profit', 'annual profit', 'profit ytd'],
        'pl_mtd': ['pl mtd', 'pl_mtd', 'pl-mtd', 'monthly profit', 'mtd profit', 'profit month'],
        'pl_qtd': ['pl qtd', 'pl_qtd', 'pl-qtd', 'quarterly profit', 'qtd profit'],
        'mv_base': ['mv base', 'mv_base', 'mv-base', 'market value base', 'base mv'],
        'mv_local': ['mv local', 'mv_local', 'mv-local', 'market value local'],
        'principal': ['principal', 'principle', 'notional', 'trade value'],
        'quantity': ['quantity', 'qty', 'amount', 'volume', 'shares'],
        'totalcash': ['total cash', 'totalcash', 'cash']
    }
    for metric, aliases in metric_aliases.items():
        for alias in aliases:
            if alias in q_lower.replace('_', ' '):
                entities['metric'] = metric
                break
        if 'metric' in entities:
            break

    if 'negative' in q_lower or re.search(r'\b<\s*0\b', q_lower) or 'below zero' in q_lower:
        entities['condition'] = 'negative'

    return entities

<strong>Chunk Builders, Summaries & Text Converters</strong>

In [24]:
def build_portfolio_summary(portfolio: str, trades_df: pd.DataFrame, holdings_df: pd.DataFrame) -> str:
    p_norm = portfolio.strip().lower()
    h = holdings_df[holdings_df['PortfolioName'].astype(str).str.lower() == p_norm]
    t = trades_df[trades_df['PortfolioName'].astype(str).str.lower() == p_norm]

    lines = []
    lines.append(f"PORTFOLIO: {portfolio}")
    lines.append(f"Total Holdings Rows: {len(h)}")
    lines.append(f"Total Trades Rows: {len(t)}")

    def sum_col(df, candidates):
        for c in candidates:
            if c in df.columns:
                return df[c].sum()
        return 0.0

    pl_ytd = sum_col(h, ['PL_YTD_num', 'PL_YTD', 'pl_ytd'])
    pl_mtd = sum_col(h, ['PL_MTD_num', 'PL_MTD', 'pl_mtd'])
    pl_qtd = sum_col(h, ['PL_QTD_num', 'PL_QTD', 'pl_qtd'])
    mv_base = sum_col(h, ['MV_Base_num', 'MV_Base', 'mv_base'])

    lines.append(f"PL_YTD_sum: {pl_ytd}")
    lines.append(f"PL_MTD_sum: {pl_mtd}")
    lines.append(f"PL_QTD_sum: {pl_qtd}")
    lines.append(f"MV_Base_sum: {mv_base}")

    lines.append("HOLDINGS SUMMARY (top securities)")
    if not h.empty:
        try:
            grouped = h.groupby('SecName').agg({
                'Qty_num': 'sum',
                'MV_Base_num': 'sum',
                'PL_YTD_num': 'sum'
            }).fillna(0).sort_values(by='MV_Base_num', ascending=False)
            for sec, row in grouped.head(20).iterrows():
                lines.append(f"{sec} | Qty={int(row.get('Qty_num',0))} | MV_Base={row.get('MV_Base_num',0):,.2f} | PL_YTD={row.get('PL_YTD_num',0):,.2f}")
        except Exception:
            for _, r in h.head(20).iterrows():
                lines.append(f"{r.get('SecName','')} | Qty={r.get('Qty','')} | Price={r.get('Price','')} | PL_YTD={r.get('PL_YTD','')}")
    else:
        lines.append("No holdings rows.")

    lines.append("TRADES SUMMARY (top trades)")
    if not t.empty:
        try:
            tgroup = t.groupby(['Name']).agg({'Quantity_num':'sum','Principal_num':'sum'}).fillna(0).sort_values(by='Principal_num', ascending=False)
            for name, row in tgroup.head(20).iterrows():
                lines.append(f"{name} | TradeQty={int(row.get('Quantity_num',0))} | Principal={row.get('Principal_num',0):,.2f}")
        except Exception:
            for _, r in t.head(20).iterrows():
                lines.append(f"{r.get('TradeTypeName','')} {r.get('Name','')} Qty={r.get('Quantity','')} Price={r.get('Price','')}")
    else:
        lines.append("No trades rows.")

    return "\n".join(lines)

In [25]:
def build_security_summary(security_id: str, trades_df: pd.DataFrame, holdings_df: pd.DataFrame) -> str:
    sid = str(security_id)
    h = holdings_df[holdings_df['SecurityId'].astype(str) == sid]
    t = trades_df[trades_df['SecurityId'].astype(str) == sid]

    lines = []
    lines.append(f"SECURITY: {sid}")
    lines.append(f"Holdings Rows: {len(h)}")
    lines.append(f"Trades Rows: {len(t)}")

    if not h.empty:
        try:
            hv = h.groupby('PortfolioName').agg({'Qty_num':'sum','MV_Base_num':'sum','PL_YTD_num':'sum'}).fillna(0)
            lines.append("--- Holdings by Portfolio ---")
            for p, row in hv.sort_values(by='MV_Base_num', ascending=False).iterrows():
                lines.append(f"{p} | Qty={int(row.get('Qty_num',0))} | MV_Base={row.get('MV_Base_num',0):,.2f} | PL_YTD={row.get('PL_YTD_num',0):,.2f}")
        except Exception:
            for _, r in h.iterrows():
                lines.append(f"{r.get('PortfolioName','')} | Qty={r.get('Qty','')} | MV_Base={r.get('MV_Base','')}")
    else:
        lines.append("No holdings rows.")

    if not t.empty:
        try:
            tv = t.groupby('TradeTypeName').agg({'Quantity_num':'sum','Principal_num':'sum'}).fillna(0)
            for tt, row in tv.iterrows():
                lines.append(f"{tt} | TotalQty={int(row.get('Quantity_num',0))} | TotalPrincipal={row.get('Principal_num',0):,.2f}")
        except Exception:
            for _, r in t.iterrows():
                lines.append(f"{r.get('TradeTypeName','')} | Qty={r.get('Quantity','')} | Principal={r.get('Principal','')}")
    else:
        lines.append("No trades rows.")

    return "\n".join(lines)

In [26]:
def trade_row_to_text(row):
    parts = []
    for k in ['PortfolioName', 'TradeTypeName', 'Name', 'Ticker', 'Quantity', 'Price', 'Principal', 'TotalCash']:
        if k in row and pd.notna(row[k]) and str(row[k]) not in ('', 'NULL'):
            parts.append(f"{k}: {row[k]}")
    return ". ".join(parts)

def holding_row_to_text(row):
    parts = []
    for k in ['PortfolioName', 'ShortName', 'SecName', 'SecurityTypeName', 'Qty', 'Price', 'MV_Base', 'PL_YTD', 'PL_MTD']:
        if k in row and pd.notna(row[k]) and str(row[k]) not in ('', 'NULL'):
            parts.append(f"{k}: {row[k]}")
    return ". ".join(parts)

In [27]:
def build_global_summary(holdings_df: pd.DataFrame) -> str:
    lines = []
    lines.append("GLOBAL SUMMARY")

    for metric in ['PL_YTD_num', 'PL_MTD_num', 'PL_QTD_num', 'MV_Base_num']:
        if metric in holdings_df.columns:
            grouped = holdings_df.groupby('PortfolioName')[metric].sum().fillna(0).sort_values(ascending=False)
            lines.append(f"Top 10 by {metric}")
            for p, v in grouped.head(10).items():
                lines.append(f"{p}: {v:,.2f}")
    return "\n".join(lines)

In [28]:
def build_all_chunks(trades_df: pd.DataFrame, holdings_df: pd.DataFrame, cache_path: Path = CHUNKS_FILE) -> List[Dict[str, Any]]:
    if cache_path.exists():
        try:
            with open(cache_path, 'rb') as f:
                chunks = pickle.load(f)
            print(f"Loaded {len(chunks)} chunks from cache.")
            return chunks
        except Exception:
            print("Failed to load chunk cache; rebuilding.")

    chunks = []
    portfolios = sorted(set(list(ALL_PORTFOLIOS)))
    for p in portfolios:
        txt = build_portfolio_summary(p, trades_df, holdings_df)
        chunks.append({'type':'portfolio', 'id':p, 'text':txt})

    sids = set()
    if 'SecurityId' in holdings_df.columns:
        sids.update([str(x) for x in holdings_df['SecurityId'].astype(str).unique() if x and x.upper() != 'NULL'])
    if 'SecurityId' in trades_df.columns:
        sids.update([str(x) for x in trades_df['SecurityId'].astype(str).unique() if x and x.upper() != 'NULL'])
    for sid in sorted(sids):
        txt = build_security_summary(sid, trades_df, holdings_df)
        chunks.append({'type':'security','id':sid,'text':txt})

    chunks.append({'type':'global','id':'global_summary','text':build_global_summary(holdings_df)})

    with open(cache_path, 'wb') as f:
        pickle.dump(chunks, f)
    print(f"Built and cached {len(chunks)} chunks.")
    return chunks

<strong>Embedding Model, FAISS Index & Retrieval</strong>

In [29]:
print("Loading embedding model...")
embed_model = SentenceTransformer(EMBED_MODEL_NAME)
print("Embedding model loaded.")

Loading embedding model...
Embedding model loaded.


In [30]:
def build_chunk_index(chunks: List[Dict[str, Any]], emb_cache: Path = CHUNK_EMB_FILE):
    if emb_cache.exists():
        try:
            with open(emb_cache, 'rb') as f:
                emb_and_texts = pickle.load(f)
            embeddings = emb_and_texts['embeddings']
            print("Loaded chunk embeddings from cache.")
        except Exception:
            print("Failed loading emb cache; recomputing.")
            embeddings = embed_model.encode([c['text'] for c in chunks], convert_to_numpy=True, show_progress_bar=True)
            with open(emb_cache, 'wb') as f:
                pickle.dump({'embeddings':embeddings}, f)
    else:
        embeddings = embed_model.encode([c['text'] for c in chunks], convert_to_numpy=True, show_progress_bar=True)
        with open(emb_cache, 'wb') as f:
            pickle.dump({'embeddings':embeddings}, f)

    emb_norm = normalize(embeddings, axis=1, norm='l2').astype('float32')
    dim = emb_norm.shape[1]
    index = faiss.IndexFlatIP(dim)
    index.add(emb_norm)
    return index, embeddings

In [31]:
all_chunks = build_all_chunks(trades, holdings)
index_chunks, chunk_embeddings = build_chunk_index(all_chunks)

Loaded 328 chunks from cache.
Loaded chunk embeddings from cache.


In [32]:
def semantic_search_chunks(question: str, top_k: int = CHUNK_TOP_K, min_score: float = CHUNK_MIN_SCORE) -> List[Dict[str, Any]]:
    q_emb = embed_model.encode([question], convert_to_numpy=True)
    q_norm = normalize(q_emb, axis=1).astype('float32')
    scores, indices = index_chunks.search(q_norm, top_k)
    results = []
    for score, idx in zip(scores[0], indices[0]):
        if idx < 0 or idx >= len(all_chunks):
            continue
        if float(score) >= min_score:
            results.append({
                'score': float(score),
                'chunk': all_chunks[int(idx)]
            })
    results = sorted(results, key=lambda x: x['score'], reverse=True)
    return results

<strong>Core Query Processing Engine</strong>

In [33]:
def filter_dataframe_fuzzy(df: pd.DataFrame, entities: Dict[str, Any], debug: bool=False) -> pd.DataFrame:
    df_filtered = df.copy()
    portfolio = entities.get('portfolio') or entities.get('portfolio_suggestion')
    if portfolio:
        pn_col = 'PortfolioName' if 'PortfolioName' in df_filtered.columns else None
        sn_col = 'ShortName' if 'ShortName' in df_filtered.columns else None
        masks = []
        if pn_col:
            masks.append(df_filtered[pn_col].astype(str).apply(lambda x: normalize_text(x) == normalize_text(portfolio)))
        if sn_col:
            masks.append(df_filtered[sn_col].astype(str).apply(lambda x: normalize_text(x) == normalize_text(portfolio)))
        if masks:
            combined = masks[0]
            for m in masks[1:]:
                combined = combined | m
            df_filtered = df_filtered[combined]
        else:
            mask = pd.Series(False, index=df_filtered.index)
            for c in df_filtered.select_dtypes(include=['object','string']).columns:
                mask = mask | df_filtered[c].astype(str).str.lower().str.contains(str(portfolio).lower(), na=False)
            df_filtered = df_filtered[mask]
        if debug:
            print(f"[debug] after portfolio filter: {len(df_filtered)} rows")

    if 'ticker' in entities and 'Ticker' in df_filtered.columns:
        t = entities['ticker']
        df_filtered = df_filtered[df_filtered['Ticker'].astype(str).apply(lambda x: normalize_text(x) == normalize_text(t))]
        if debug:
            print(f"[debug] after ticker filter: {len(df_filtered)} rows")

    if 'trade_type' in entities and 'TradeTypeName' in df_filtered.columns:
        df_filtered = df_filtered[df_filtered['TradeTypeName'].astype(str).str.lower() == entities['trade_type'].lower()]
        if debug:
            print(f"[debug] after trade_type filter: {len(df_filtered)} rows")

    if entities.get('condition') == 'negative':
        pl_col = None
        for candidate in ['PL_YTD_num','PL_MTD_num','PL_QTD_num']:
            if candidate in df_filtered.columns:
                pl_col = candidate
                break
        if pl_col:
            df_filtered = df_filtered[df_filtered[pl_col] < 0]
            if debug:
                print(f"[debug] after negative condition: {len(df_filtered)} rows")

    return df_filtered

In [34]:
def exec_count(dataset: str, entities: Dict[str, Any], debug: bool=False) -> Tuple[int, pd.DataFrame]:
    df = trades if dataset == 'trades' else holdings
    df_filtered = filter_dataframe_fuzzy(df, entities, debug=debug)
    return len(df_filtered), df_filtered

def exec_aggregate(dataset: str, metric: str, entities: Dict[str, Any], agg='sum', debug: bool=False):
    df = trades if dataset == 'trades' else holdings
    df_filtered = filter_dataframe_fuzzy(df, entities, debug=debug)
    if df_filtered.empty:
        return None, df_filtered

    metric_map = {
        'principal': 'Principal_num',
        'totalcash': 'TotalCash_num',
        'quantity': 'Quantity_num',
        'pl_ytd': 'PL_YTD_num',
        'pl_mtd': 'PL_MTD_num',
        'pl_qtd': 'PL_QTD_num',
        'mv_base': 'MV_Base_num',
        'mv_local': 'MV_Local_num',
        'qty': 'Qty_num',
    }
    col = metric_map.get(metric)
    if not col or col not in df_filtered.columns:
        for cand in metric_map.values():
            if cand in df_filtered.columns:
                col = cand
                break
        if not col:
            return None, df_filtered

    if agg == 'sum':
        result = df_filtered[col].sum()
    elif agg == 'mean':
        result = df_filtered[col].mean()
    else:
        result = df_filtered[col].sum()

    return float(result) if not pd.isna(result) else None, df_filtered

def exec_rank(metric='PL_YTD', top_n=5, ascending=False):
    col = metric + '_num'
    if col not in holdings.columns:
        possible = [c for c in holdings.columns if c.lower().startswith(metric.lower()) and c.endswith('_num')]
        if possible:
            col = possible[0]
        else:
            return None
    name_col = 'PortfolioName' if 'PortfolioName' in holdings.columns else 'ShortName'
    grouped = holdings.groupby(name_col)[col].sum().fillna(0)
    sorted_series = grouped.sort_values(ascending=ascending)
    return sorted_series.head(top_n)

In [35]:
def call_llm_formatter(question: str, summary: str, evidence_text: str = "", max_tokens: int = 300) -> str:
    if not OPENROUTER_API_KEY:
        if summary is None or summary == "":
            return "Sorry can not find the answer"
        if summary.startswith("Count:"):
            num = summary.split(":",1)[1].strip()
            return f"There are {num} matching rows."
        if summary.startswith("Total"):
            return summary.replace("Total ", "The total ").replace("_", " ")
        if summary.startswith("Found"):
            return summary
        return summary

    system_prompt = """
        You are a strict financial results formatter. 
        You MUST follow the rules below with zero exceptions.

        ABSOLUTE RULES:
        1. You are NOT allowed to calculate, infer, assume, convert, or transform numbers.
        2. You may ONLY restate the information EXACTLY as it appears in the SUMMARY.
        3. DO NOT use or reference EVIDENCE ROWS for facts or numbers. They are ONLY context.
        4. DO NOT add missing numbers, names, rankings, or conclusions.
        5. DO NOT guess portfolio names, totals, rankings, currency values, or trade amounts.
        6. ANY number you mention MUST appear verbatim in the SUMMARY.
        7. When referring to currency, ALWAYS use “$” — 
        but DO NOT add currency symbols or convert the number.
        Example:
            SUMMARY: "Total PL_YTD: -50000.25"
            OUTPUT: "The total PL YTD is $-50000.25."
        8. If the SUMMARY contains wording indicating no data (“No data”, “None”, “Empty”)
        OR if SUMMARY is blank, respond EXACTLY with:
        “Sorry can not find the answer”
        9. If the SUMMARY describes a list, ranking, count, or total,
        you may rewrite it into natural language but MUST keep meaning 100% unchanged.
        10. ALL outputs must be short, factual, and contain ZERO speculation.

        ILLEGAL ACTIONS (NEVER DO THESE):
        - Creating numbers not in SUMMARY
        - Inferring best/worst portfolios
        - Using EVIDENCE to compute missing financial metrics
        - Expanding answers with extra details
        - Adding currency formatting, commas, $ signs, or converting units

        If any rule is violated, the answer is invalid.
        """


    user_message = f"""
        QUESTION:
        {question}

        SUMMARY (the ONLY facts you may use):
        {summary}

        EVIDENCE (context ONLY — DO NOT extract facts from this):
        {evidence_text}

        TASK:
        Rewrite ONLY the SUMMARY into a clear English sentence.
        - You MUST use “$” for any financial amount.
        - You MUST NOT invent or modify numbers.
        - You MUST NOT compute or infer anything.
        - If SUMMARY shows no data or is empty, reply EXACTLY:
            "Sorry can not find the answer".
        """

    payload = {
        "model": "openai/gpt-4o-mini",
        "messages": [
            {"role":"system","content": system_prompt},
            {"role":"user","content": user_message}
        ],
        "temperature": 0.05,
        "max_tokens": max_tokens
    }
    
    try:
        r = requests.post(
            "https://openrouter.ai/api/v1/chat/completions",
            headers={
                "Authorization": f"Bearer {OPENROUTER_API_KEY}",
                "Content-Type": "application/json"
            },
            json=payload,
            timeout=30
        )
        r.raise_for_status()
        resp = r.json()
        return resp['choices'][0]['message']['content'].strip()
    except Exception as e:
        print("OpenRouter call failed:", str(e))
        if summary is None or summary == "":
            return "Sorry can not find the answer"
        return summary

In [36]:
def format_evidence_rows(results: List[Dict[str, Any]], max_rows:int=6) -> str:
    pieces = []
    for i, r in enumerate(results[:max_rows]):
        c = r['chunk']
        short_text = c["text"][:250].replace("\n", " | ")
        pieces.append(
            f"[{i+1}] ({c['type']}:{c['id']}) score={r['score']:.3f} -> {short_text}"
        )
    return "\n".join(pieces)

In [37]:
def classify_query(question: str) -> Tuple[str, str]:
    q = (question or "").lower()
    dataset = None
    if any(w in q for w in ['trade', 'trades', 'buy', 'sell', 'traded', 'counterparty', 'principal']):
        dataset = 'trades'
    if any(w in q for w in ['holding', 'holdings', 'fund', 'portfolio', 'performed', 'performance', 'pl', 'profit', 'loss', 'mv']):
        dataset = 'both' if dataset == 'trades' else 'holdings'
    if dataset is None:
        dataset = 'both'

    operation = None
    if any(w in q for w in ['how many', 'count', 'number of', 'total number']):
        operation = 'count'
    elif any(w in q for w in ['total', 'sum', 'aggregate']) and any(w in q for w in ['principal', 'quantity', 'cash', 'pl', 'mv', 'qty']):
        operation = 'aggregate'
    elif any(w in q for w in ['best', 'worst', 'top', 'rank', 'performed', 'highest', 'lowest']):
        operation = 'rank'
    elif any(w in q for w in ['show']) and not any(w in q for w in ['profit', 'pl', 'mtd', 'ytd']):
        operation = 'show'
    else:
        operation = 'unknown'
    return dataset, operation

In [38]:
def answer_question(question: str, debug: bool=False) -> str:
    if not question or not str(question).strip():
        return "Sorry can not find the answer"

    entities = extract_entities_fuzzy(question, debug=debug)
    dataset, operation = classify_query(question)

    if debug:
        print("[debug] entities:", entities)
        print("[debug] dataset, operation:", dataset, operation)

    if operation == 'count':
        datasets_try = []
        if dataset in ('trades','both'):
            datasets_try.append('trades')
        if dataset in ('holdings','both'):
            datasets_try.append('holdings')

        for ds in datasets_try:
            count, df_filtered = exec_count(ds, entities, debug=debug)
            if count > 0:
                evidence = semantic_search_chunks(question, top_k=CHUNK_TOP_K, min_score=CHUNK_MIN_SCORE)
                summary = f"Count: {count} {ds}"
                evidence_text = format_evidence_rows(evidence)
                return call_llm_formatter(question, summary, evidence_text)
        return "Sorry can not find the answer"

    if operation == 'aggregate':
        metric = entities.get('metric')
        if not metric:
            return "Sorry can not find the answer"
        ds_target = 'trades' if metric in ('principal','totalcash','quantity') else 'holdings'
        result, df_filtered = exec_aggregate(ds_target, metric, entities, agg='sum', debug=debug)
        if result is None:
            return "Sorry can not find the answer"
        evidence = semantic_search_chunks(question, top_k=CHUNK_TOP_K, min_score=CHUNK_MIN_SCORE)
        summary = f"Total {metric}: {result:,.2f}"
        evidence_text = format_evidence_rows(evidence)
        return call_llm_formatter(question, summary, evidence_text)

    if operation == 'rank':
        ascending = 'worst' in question.lower() or 'lowest' in question.lower()
        metric = entities.get('metric', 'pl_ytd')
        if metric not in ('pl_ytd','pl_mtd','pl_qtd','mv_base','mv_local'):
            metric = 'pl_ytd'
        top_n = 5
        m = re.search(r'top\s*(\d+)', question.lower())
        if m:
            top_n = int(m.group(1))
        ranked = exec_rank(metric.upper(), top_n=top_n, ascending=ascending)
        if ranked is None or ranked.empty:
            return "Sorry can not find the answer"
        lines = [f"Ranking by {metric.upper()}:"]
        for i, (portfolio, value) in enumerate(ranked.items(), 1):
            lines.append(f"{i}. {portfolio}: {value:,.2f}")
        summary = "\n".join(lines)
        top_portfolio = ranked.index[0]
        evidence = semantic_search_chunks(f"{top_portfolio} {metric}", top_k=3, min_score=CHUNK_MIN_SCORE)
        evidence_text = format_evidence_rows(evidence)
        return call_llm_formatter(question, summary, evidence_text)

    if operation == 'show':
        strong_filter_present = any(k in entities for k in ['portfolio','ticker','trade_type','condition'])
        results = []
        if strong_filter_present:
            if dataset in ('trades','both'):
                _, df_filtered_trades = exec_count('trades', entities)
                if not df_filtered_trades.empty:
                    for i, row in df_filtered_trades.head(20).iterrows():
                        results.append({'score': 1.0, 'chunk': {'type':'trades_row','id':int(i),'text': str(row.to_dict())}})
            if dataset in ('holdings','both'):
                _, df_filtered_hold = exec_count('holdings', entities)
                if not df_filtered_hold.empty:
                    for i, row in df_filtered_hold.head(20).iterrows():
                        results.append({'score': 1.0, 'chunk': {'type':'holdings_row','id':int(i),'text': str(row.to_dict())}})
            if results:
                summary = f"Found {len(results)} relevant rows (showing top {min(10,len(results))})"
                evidence_text = format_evidence_rows(results)
                return call_llm_formatter(question, summary, evidence_text)

        results = semantic_search_chunks(question, top_k=10, min_score=0.30)
        if not results:
            return "Sorry can not find the answer"
        summary = f"Found {len(results)} relevant chunks (showing top {min(6,len(results))})"
        evidence_text = format_evidence_rows(results)
        return call_llm_formatter(question, summary, evidence_text)

    for ds in (['trades'] if dataset == 'trades' else (['holdings'] if dataset == 'holdings' else ['trades','holdings'])):
        count, df_filtered = exec_count(ds, entities)
        if count > 0:
            summary = f"Found {count} matching rows in {ds}"
            evidence = semantic_search_chunks(question, top_k=CHUNK_TOP_K, min_score=CHUNK_MIN_SCORE)
            evidence_text = format_evidence_rows(evidence)
            return call_llm_formatter(question, summary, evidence_text)

    results = semantic_search_chunks(question, top_k=10, min_score=0.30)
    if not results:
        return "Sorry can not find the answer"
    summary = f"Found {len(results)} relevant results"
    evidence_text = format_evidence_rows(results)
    return call_llm_formatter(question, summary, evidence_text)

<strong>TEST RUN</strong>

In [39]:
test_questions = [
    "Which fund has best quarterly profit?",
    "Which portfolio performed best based on PL_MTD?",
    "Total trades for GreenRock Capital?",
    "Which fund has worst yearly PL?",
    "Which portfolio holds the most of SecurityId 288800?",
    "Show all trades for SecurityId 290081?",   
    "Total holdings for Garfield",
    "Total PL YTD for Garfield",
    "Which fund has best yearly profit?",
    "Total quantity of AA stock traded?",
    "Which fund performed best based on PL_YTD?",
    "Show top 3 funds by profit",
    "Show holdings with negative PL_YTD",
    "Who is the CEO?",
    "Total holdings for Sandberg Fund",
    "Total PL YTD for Wellington Portfolio",
    "Total quantity of TSLA stock traded?",
    "Show top 3 portfolios by market value",
    "Show holdings with negative PL_QTD",
    "Total MV Base for Redwood Income Fund",
    "Count all FX trades for USD/JPY",
    "What is the total MV Base for Heather?",
    "Show the top 5 securities in MNC Investment Fund by market value.",
    "Total PL MTD for Opium Holdings Partners?",
    "Total PL QTD for Northpoint 401K?",
    "Total holdings quantity for SecurityId 288790?",
    "Total traded quantity for SecurityId 288790?",
    "Total principal traded for SecurityId 290067?",
    "Which counterparty handled the most trades?",
    "How many total Buy trades were executed across all portfolios?"
]


In [40]:
print("=" * 80)
print("HYBRID RAG CHATBOT TEST RESULTS")
print("=" * 80)

for i, q in enumerate(test_questions, 1):
    print(f"\n[Q{i}] {q}")
    answer = answer_question(q, debug=False)
    print(f"[A{i}] {answer}")
    print("-" * 80)

HYBRID RAG CHATBOT TEST RESULTS

[Q1] Which fund has best quarterly profit?
[A1] The ranking by PL QTD shows that Ytum has the highest profit at $1,493,081.48, followed by Platpot at $1,489,647.46, Opium Holdings Partners at $513,265.37, NPSMF1 at $46,415.46, and NPSMF2 at $45,402.83.
--------------------------------------------------------------------------------

[Q2] Which portfolio performed best based on PL_MTD?
[A2] The ranking by PL MTD shows that Hi Yield performed best with $26,014.79, followed by Warren Lee IG with $20,001.85, Platpot with $11,825.07, Ytum with $6,487.80, and IG Corp with $0.00.
--------------------------------------------------------------------------------

[Q3] Total trades for GreenRock Capital?
[A3] Sorry can not find the answer.
--------------------------------------------------------------------------------

[Q4] Which fund has worst yearly PL?
[A4] The fund with the worst yearly PL is CoYold 7 with a PL YTD of $-5,000,000,000.00.
---------------------

<strong>CUSTOM QUERY RUN</strong>

In [ ]:
# Please make sure to paste your openrouter api link before running your custom query !!!
# you can view .env.example to create a .env file in a similar structure.


query = "____WRITE YOUR QUERY HERE_____"
answer = answer_question(query, debug=False)
print(answer)